In [1]:
#PYTHON
import h5py
import numpy as np
import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pyspark

#SPARK
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import *
from operator import add

Spark Configuration

In [2]:
# Spark Session - 2 CORE - imporvement CONFIGURATION!!
spark_session = SparkSession.builder\
    .master("local[2]")\
    .appName("pseudo_spark_nora")\
    .getOrCreate()

# RDD API
spark_context = spark_session.sparkContext
spark_context.setLogLevel("INFO")

sqlContext = SQLContext(spark_session.sparkContext)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/01 12:53:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/ubuntu/.local/lib/python3.8/site-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


File Path

In [3]:
#file_roots = ['hdfs://130.238.29.183:9000//input/millionsongsubset.tar.gz']
#directory_path = "data/B/A/A"
directory_path = "data_AZ"

# 1) Saving the paths to the HDF5 files

In [4]:
# GET .H5 FILENAMES
h5_file_paths = []
for root, _, files in os.walk(directory_path):
    for name in files:
        h5_file_paths.append(os.path.join(root, name))

print('{} filenames gathered (PYTHON)'.format(len(h5_file_paths)))

319 filenames gathered (PYTHON)


# 2) List h5_file_paths is converted to RDD

In [5]:
h5_file_paths

['data_AZ/A/Z/TRAAZQW128F93430DC.h5',
 'data_AZ/A/Z/TRAAZIE128F427961E.h5',
 'data_AZ/A/Z/TRAAZBD128F427A97D.h5',
 'data_AZ/A/Z/TRAAZBX128F1492AF8.h5',
 'data_AZ/A/Z/TRAAZKV12903CDDFE0.h5',
 'data_AZ/A/Z/TRAAZLK128F93586D6.h5',
 'data_AZ/A/Z/TRAAZOZ12903C9CABF.h5',
 'data_AZ/A/Z/TRAAZJU12903C980B6.h5',
 'data_AZ/A/Z/TRAAZGC12903CB5DFB.h5',
 'data_AZ/A/Z/TRAAZHO128F427C9EC.h5',
 'data_AZ/A/Z/TRAAZQF128F4285BC3.h5',
 'data_AZ/A/Z/TRAAZYI12903CB2EE4.h5',
 'data_AZ/A/Z/TRAAZPJ128F4292DB5.h5',
 'data_AZ/A/C/TRAACOW128F933E35F.h5',
 'data_AZ/A/C/TRAACPE128F421C1B9.h5',
 'data_AZ/A/C/TRAACSL128F93462F4.h5',
 'data_AZ/A/C/TRAACIW12903CC0F6D.h5',
 'data_AZ/A/C/TRAACNS128F14A2DF5.h5',
 'data_AZ/A/C/TRAACFV128F935E50B.h5',
 'data_AZ/A/C/TRAACCG128F92E8A55.h5',
 'data_AZ/A/C/TRAACER128F4290F96.h5',
 'data_AZ/A/C/TRAACVS128E078BE39.h5',
 'data_AZ/A/C/TRAACQT128F9331780.h5',
 'data_AZ/A/C/TRAACLV128F427E123.h5',
 'data_AZ/A/C/TRAACZK128F4243829.h5',
 'data_AZ/A/C/TRAACHN128F1489601.h5',
 'data_AZ/A/

In [6]:
file_paths = spark_context.parallelize(h5_file_paths)
file_paths

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [7]:
file_first = file_paths.first()
file_first

24/03/01 12:54:00 INFO SparkContext: Starting job: runJob at PythonRDD.scala:181
24/03/01 12:54:00 INFO DAGScheduler: Got job 0 (runJob at PythonRDD.scala:181) with 1 output partitions
24/03/01 12:54:00 INFO DAGScheduler: Final stage: ResultStage 0 (runJob at PythonRDD.scala:181)
24/03/01 12:54:00 INFO DAGScheduler: Parents of final stage: List()
24/03/01 12:54:00 INFO DAGScheduler: Missing parents: List()
24/03/01 12:54:00 INFO DAGScheduler: Submitting ResultStage 0 (PythonRDD[1] at RDD at PythonRDD.scala:53), which has no missing parents
24/03/01 12:54:00 INFO MemoryStore: Block broadcast_0 stored as values in memory (estimated size 5.8 KiB, free 413.9 MiB)
24/03/01 12:54:00 INFO MemoryStore: Block broadcast_0_piece0 stored as bytes in memory (estimated size 3.8 KiB, free 413.9 MiB)
24/03/01 12:54:00 INFO BlockManagerInfo: Added broadcast_0_piece0 in memory on host-192-168-2-34-de1:40527 (size: 3.8 KiB, free: 413.9 MiB)
24/03/01 12:54:00 INFO SparkContext: Created broadcast 0 from br

'data_AZ/A/Z/TRAAZQW128F93430DC.h5'

# 3) H5Py file object file creation from the H5 file 

In [8]:
def extract_info(file_path):
    with h5py.File(file_path, 'r') as file:
        # Informationen extrahieren
        song = file['analysis']['songs']
        np_song = np.array(song[0])
        selected_features = np_song[['danceability', 'duration', 'tempo', 'energy', 'loudness']].ravel()
        return selected_features.tolist()

In [9]:
#file = h5py.File(file_first)
files_infos_rdd = file_paths.map(extract_info)

In [11]:
print(files_infos_rdd.collect())

24/03/01 12:54:10 INFO SparkContext: Starting job: collect at /tmp/ipykernel_56411/3743736607.py:1
24/03/01 12:54:10 INFO DAGScheduler: Got job 2 (collect at /tmp/ipykernel_56411/3743736607.py:1) with 2 output partitions
24/03/01 12:54:10 INFO DAGScheduler: Final stage: ResultStage 2 (collect at /tmp/ipykernel_56411/3743736607.py:1)
24/03/01 12:54:10 INFO DAGScheduler: Parents of final stage: List()
24/03/01 12:54:10 INFO DAGScheduler: Missing parents: List()
24/03/01 12:54:10 INFO DAGScheduler: Submitting ResultStage 2 (PythonRDD[2] at collect at /tmp/ipykernel_56411/3743736607.py:1), which has no missing parents
24/03/01 12:54:10 INFO MemoryStore: Block broadcast_2 stored as values in memory (estimated size 5.9 KiB, free 413.9 MiB)
24/03/01 12:54:10 INFO MemoryStore: Block broadcast_2_piece0 stored as bytes in memory (estimated size 4.0 KiB, free 413.9 MiB)
24/03/01 12:54:10 INFO BlockManagerInfo: Added broadcast_2_piece0 in memory on host-192-168-2-34-de1:40527 (size: 4.0 KiB, free:

[[(0.0, 224.60036, 210.871, 0.0, -13.256)], [(0.0, 167.20934, 196.034, 0.0, -7.928)], [(0.0, 266.4224, 80.149, 0.0, -5.947)], [(0.0, 111.0722, 82.422, 0.0, -13.826)], [(0.0, 356.41424, 130.861, 0.0, -7.598)], [(0.0, 260.28363, 90.008, 0.0, -2.939)], [(0.0, 256.39138, 122.174, 0.0, -8.632)], [(0.0, 240.61342, 64.401, 0.0, -14.367)], [(0.0, 313.7824, 107.197, 0.0, -11.488)], [(0.0, 265.37751, 124.177, 0.0, -9.28)], [(0.0, 252.57751, 75.955, 0.0, -7.605)], [(0.0, 214.12526, 105.196, 0.0, -7.225)], [(0.0, 316.49914, 95.981, 0.0, -14.544)], [(0.0, 156.39465, 95.957, 0.0, -18.051)], [(0.0, 342.56934, 110.189, 0.0, -11.939)], [(0.0, 164.80608, 130.1, 0.0, -10.922)], [(0.0, 338.31138, 125.006, 0.0, -6.486)], [(0.0, 168.25424, 127.025, 0.0, -9.616)], [(0.0, 124.86485, 137.522, 0.0, -12.332)], [(0.0, 269.58322, 111.874, 0.0, -8.05)], [(0.0, 207.77751, 166.862, 0.0, -4.264)], [(0.0, 173.66159, 41.993, 0.0, -7.762)], [(0.0, 484.62322, 138.007, 0.0, -15.335)], [(0.0, 186.48771, 90.022, 0.0, -4.516)

24/03/01 12:54:11 INFO TaskSetManager: Finished task 1.0 in stage 2.0 (TID 4) in 802 ms on host-192-168-2-34-de1 (executor driver) (2/2)
24/03/01 12:54:11 INFO DAGScheduler: ResultStage 2 (collect at /tmp/ipykernel_56411/3743736607.py:1) finished in 0.848 s
24/03/01 12:54:11 INFO DAGScheduler: Job 2 is finished. Cancelling potential speculative or zombie tasks for this job
24/03/01 12:54:11 INFO TaskSchedulerImpl: Removed TaskSet 2.0, whose tasks have all completed, from pool 
24/03/01 12:54:11 INFO TaskSchedulerImpl: Killing all running tasks in stage 2: Stage finished
24/03/01 12:54:11 INFO DAGScheduler: Job 2 finished: collect at /tmp/ipykernel_56411/3743736607.py:1, took 0.858435 s


# 4) Create Spark Data Frame

In [12]:
# Definiere das Schema für den DataFrame
schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("duration", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("loudness", FloatType(), True),
])

In [13]:
spark_df = sqlContext.createDataFrame(files_infos_rdd.flatMap(lambda x: x), schema=schema)

24/03/01 12:54:15 INFO BlockManagerInfo: Removed broadcast_0_piece0 on host-192-168-2-34-de1:40527 in memory (size: 3.8 KiB, free: 413.9 MiB)
24/03/01 12:54:15 INFO BlockManagerInfo: Removed broadcast_2_piece0 on host-192-168-2-34-de1:40527 in memory (size: 4.0 KiB, free: 413.9 MiB)
24/03/01 12:54:15 INFO SharedState: Setting hive.metastore.warehouse.dir ('null') to the value of spark.sql.warehouse.dir.
24/03/01 12:54:15 INFO SharedState: Warehouse path is 'file:/home/ubuntu/dataeng1/Project/spark-warehouse'.


In [14]:
# Zeige den DataFrame-Inhalt
spark_df.show()

24/03/01 12:54:19 INFO CodeGenerator: Code generated in 494.50861 ms
24/03/01 12:54:19 INFO SparkContext: Starting job: showString at NativeMethodAccessorImpl.java:0
24/03/01 12:54:20 INFO DAGScheduler: Got job 3 (showString at NativeMethodAccessorImpl.java:0) with 1 output partitions
24/03/01 12:54:20 INFO DAGScheduler: Final stage: ResultStage 3 (showString at NativeMethodAccessorImpl.java:0)
24/03/01 12:54:20 INFO DAGScheduler: Parents of final stage: List()
24/03/01 12:54:20 INFO DAGScheduler: Missing parents: List()
24/03/01 12:54:20 INFO DAGScheduler: Submitting ResultStage 3 (MapPartitionsRDD[8] at showString at NativeMethodAccessorImpl.java:0), which has no missing parents
24/03/01 12:54:20 INFO MemoryStore: Block broadcast_3 stored as values in memory (estimated size 22.3 KiB, free 413.9 MiB)
24/03/01 12:54:20 INFO MemoryStore: Block broadcast_3_piece0 stored as bytes in memory (estimated size 10.4 KiB, free 413.9 MiB)
24/03/01 12:54:20 INFO BlockManagerInfo: Added broadcast_3

+------------+---------+-------+------+--------+
|danceability| duration|  tempo|energy|loudness|
+------------+---------+-------+------+--------+
|         0.0|224.60036|210.871|   0.0| -13.256|
|         0.0|167.20934|196.034|   0.0|  -7.928|
|         0.0| 266.4224| 80.149|   0.0|  -5.947|
|         0.0| 111.0722| 82.422|   0.0| -13.826|
|         0.0|356.41425|130.861|   0.0|  -7.598|
|         0.0|260.28363| 90.008|   0.0|  -2.939|
|         0.0| 256.3914|122.174|   0.0|  -8.632|
|         0.0|240.61342| 64.401|   0.0| -14.367|
|         0.0| 313.7824|107.197|   0.0| -11.488|
|         0.0| 265.3775|124.177|   0.0|   -9.28|
|         0.0|252.57751| 75.955|   0.0|  -7.605|
|         0.0|214.12526|105.196|   0.0|  -7.225|
|         0.0|316.49915| 95.981|   0.0| -14.544|
|         0.0|156.39465| 95.957|   0.0| -18.051|
|         0.0|342.56934|110.189|   0.0| -11.939|
|         0.0|164.80608|  130.1|   0.0| -10.922|
|         0.0|338.31137|125.006|   0.0|  -6.486|
|         0.0|168.25

24/03/01 12:54:23 INFO CodeGenerator: Code generated in 72.622655 ms


# Calculation (Average)

In [15]:
average_values = spark_df.agg({'danceability': 'avg', 'duration': 'avg', 'tempo': 'avg', 'energy': 'avg', 'loudness': 'avg'}).collect()[0]
print("Average Values::")
for col_name, avg_value in average_values.asDict().items():
    print(f"{col_name}: {avg_value}")

24/03/01 12:54:24 INFO CodeGenerator: Code generated in 159.97677 ms
24/03/01 12:54:24 INFO DAGScheduler: Registering RDD 10 (collect at /tmp/ipykernel_56411/3998951206.py:1) as input to shuffle 0
24/03/01 12:54:24 INFO DAGScheduler: Got map stage job 4 (collect at /tmp/ipykernel_56411/3998951206.py:1) with 2 output partitions
24/03/01 12:54:24 INFO DAGScheduler: Final stage: ShuffleMapStage 4 (collect at /tmp/ipykernel_56411/3998951206.py:1)
24/03/01 12:54:24 INFO DAGScheduler: Parents of final stage: List()
24/03/01 12:54:24 INFO DAGScheduler: Missing parents: List()
24/03/01 12:54:24 INFO DAGScheduler: Submitting ShuffleMapStage 4 (MapPartitionsRDD[10] at collect at /tmp/ipykernel_56411/3998951206.py:1), which has no missing parents
24/03/01 12:54:24 INFO MemoryStore: Block broadcast_4 stored as values in memory (estimated size 36.3 KiB, free 413.9 MiB)
24/03/01 12:54:24 INFO MemoryStore: Block broadcast_4_piece0 stored as bytes in memory (estimated size 15.5 KiB, free 413.9 MiB)
24

Average Values::
avg(duration): 236.8841377784466
avg(tempo): 125.78977758234197
avg(energy): 0.0
avg(danceability): 0.0
avg(loudness): -10.435855814655746


# STOP SPARK

In [16]:
# release the cores
spark_context.stop()

24/03/01 12:54:28 INFO SparkContext: SparkContext is stopping with exitCode 0.
24/03/01 12:54:28 INFO SparkUI: Stopped Spark web UI at http://host-192-168-2-34-de1:4040
24/03/01 12:54:28 INFO MapOutputTrackerMasterEndpoint: MapOutputTrackerMasterEndpoint stopped!
24/03/01 12:54:28 INFO MemoryStore: MemoryStore cleared
24/03/01 12:54:28 INFO BlockManager: BlockManager stopped
24/03/01 12:54:28 INFO BlockManagerMaster: BlockManagerMaster stopped
24/03/01 12:54:28 INFO OutputCommitCoordinator$OutputCommitCoordinatorEndpoint: OutputCommitCoordinator stopped!
24/03/01 12:54:28 INFO SparkContext: Successfully stopped SparkContext
